### Data Shuffle

- Its potentially one of the most performant consuming operation in data distributed systems:
    - Network I/O is slow
    - Memory pressure
    - Disk spills

- Shuffling:
    - Redistributing data across the cluster
    - Occurs when data needs to move between executors, i.e. when joining two data sets
    - `groupByKey` `reduceByKey` `Joins`
    - Data Skew, uneven distribution of data
    - Partitioning
    - Caching
    - Data locality

- `Narrow transformation`: are operations where each partition's data can be processed independently (map, filter, flatmap)
- `Wide transformations`: are operations that require data from multiple partitions to be combined (join, groupBy, distinct, reduceByKey)

- Technique to minimize Data Shuffling:
    - Repartition before shuffling: `df = df.partition(<number-of-partitions-OR-list-of-columns>)`
    - Broadcast joins: `df_large.join(broadcast(df_small), "key")`
    - Caching data: `df.cache()` (multiple joins for different tables)


<img src="../img/data_distribution.png" alt="Data distribution" width="750">

In [ ]:
from pyspark.sql import SparkSession

In [4]:
# Initialize Spark session
spark = SparkSession.builder.appName("ShuffleExample").getOrCreate()

# Set log level to ERROR to reduce verbosity
spark.sparkContext.setLogLevel("ERROR")

In [5]:
# Create a DataFrame with sample data
data = [
    (1, "Alice", 100),
    (2, "Bob", 200),
    (3, "Alice", 150),
    (4, "Bob", 250),
    (5, "Charlie", 300)
]

columns = ["id", "name", "amount"]
df = spark.createDataFrame(data, columns)

In [6]:
# Group by "name" and sum the "amount" to force a shuffle
grouped_df = df.groupBy("name").sum("amount")

In [7]:
# Use "extended" to get detailed plan
grouped_df.explain(mode="extended")

== Parsed Logical Plan ==
'Aggregate ['name], ['name, sum(amount#2L) AS sum(amount)#10L]
+- LogicalRDD [id#0L, name#1, amount#2L], false

== Analyzed Logical Plan ==
name: string, sum(amount): bigint
Aggregate [name#1], [name#1, sum(amount#2L) AS sum(amount)#10L]
+- LogicalRDD [id#0L, name#1, amount#2L], false

== Optimized Logical Plan ==
Aggregate [name#1], [name#1, sum(amount#2L) AS sum(amount)#10L]
+- Project [name#1, amount#2L]
   +- LogicalRDD [id#0L, name#1, amount#2L], false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[name#1], functions=[sum(amount#2L)], output=[name#1, sum(amount)#10L])
   +- Exchange hashpartitioning(name#1, 200), ENSURE_REQUIREMENTS, [plan_id=15]
      +- HashAggregate(keys=[name#1], functions=[partial_sum(amount#2L)], output=[name#1, sum#14L])
         +- Project [name#1, amount#2L]
            +- Scan ExistingRDD[id#0L,name#1,amount#2L]



In [8]:
# Trigger execution (e.g., show results)
grouped_df.show()

+-------+-----------+
|   name|sum(amount)|
+-------+-----------+
|  Alice|        250|
|    Bob|        450|
|Charlie|        300|
+-------+-----------+



In [9]:
#Stop Spark Session
spark.stop()